In [ ]:
import tensorflow as tf
import pathlib
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import regularizers

In [ ]:
x_train = pathlib.Path("/content/hotdog-nothotdog/train")
x_test = pathlib.Path("/content/hotdog-nothotdog/test")
train_ds = tf.keras.utils.image_dataset_from_directory(
    x_train,
    seed=123,
    image_size =(299,299),
    batch_size = 32
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    x_test,
    seed=123,
    image_size=(299,299),
    batch_size = 32
)
print(train_ds.class_names)

In [ ]:
train_gen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_gen  = ImageDataGenerator(rescale = 1.0/255.)
x_train = pathlib.Path("/content/hotdog-nothotdog/train")
x_test = pathlib.Path("/content/hotdog-nothotdog/test")
train_ds = train_gen.flow_from_directory(x_train,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(299, 299))     
test_ds =  test_gen.flow_from_directory(x_test,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(299, 299))     

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(299, 299,3)),
  tf.keras.layers.Conv2D(32,(3,3), activation = 'relu'),
  tf.keras.layers.MaxPooling2D((2,2)),
  tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
  tf.keras.layers.MaxPooling2D((2,2)),
  tf.keras.layers.Conv2D(128,(3,3), activation = 'relu'),
  tf.keras.layers.MaxPooling2D((2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(150, activation='relu',kernel_regularizer=regularizers.l2(0.01),),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2,activation="sigmoid")
])

In [ ]:
predictions = model(train_ds[:1]).numpy()
predictions

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
loss_fn(test_ds[:1], predictions).numpy()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_ds, validation_data=test_ds, epochs=30)

In [ ]:

model.save('my_model.keras')